In [1]:
import pickle
import copy
import fiona

import alphashape as ashp
import pandas as pd
import numpy as np
import colorcet as cc
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import LineString, Polygon, Point, MultiPoint
from shapely.ops import transform, unary_union
from shapely import affinity
from descartes import PolygonPatch
from scipy.optimize import minimize
from multiprocessing import Pool
from tqdm import tqdm
from matplotlib import colors
from pyproj import Proj
from scipy.interpolate import griddata

In [2]:
plt.style.use('dark_background')
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [3]:
data_maps = {
    'f12': {
        '072016': ['2388', '6088'],
        '072019': ['2388', '6088']
    },
    'f10': {
        '072014': ['6088', '6130'],
        '072017': ['2388', '6088']
    },
    'f7': {
        '072014': ['6088', '6130'],
        '072018': ['7130', '8240']
    },
    'f11': {
        '072014': ['6088', '6130'],
        '072017': ['6130', '7130']
    },
    'f8': {
        '072017': ['2388', '6088'],
        '072019': ['2388', '6088']
    },
    'f1': {
        '072017': ['7130', '6130'],
        '072019': ['8250', '7130']
    },
    'f2': {
        '072017': ['7130', '6130'],
        '072019': ['8250', '7130']
    }
}

In [4]:
OPT_PICKLE_PATH = '/home/yang/output/eswe/opt/'
ESW_PICKLE_PATH = '/home/yang/output/eswe/esw/'
REPORT_PATH = '/home/yang/output/eswe/rpt/'
OUTPUT_PATH = '/home/yang/output/eswe/eff-rev/'
EVENT = 'jkwh'

In [5]:
# Load the boundary
fss = gpd.read_file('/home/yang/data/raw/jkwh.kml', driver='KML')

/home/yang/anaconda3/envs/sashimi/lib/python3.8/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [6]:
def compute_time_stats(d0,d1):
    # Get the time under each label
    tf0 = len(d0) / 3600
    tw0 = len(d0[d0['mode'] == 'w']) / 3600
    tp0 = len(d0[d0['mode'] == 'nw-p']) / 3600
    ts0 = len(d0[d0['mode'] == 'nw-s']) / 3600
    tr0 = len(d0[d0['mode'] == 'nw-t']) / 3600
    tf1 = len(d1) / 3600
    tw1 = len(d1[d1['mode'] == 'w']) / 3600
    tp1 = len(d1[d1['mode'] == 'nw-p']) / 3600
    ts1 = len(d1[d1['mode'] == 'nw-s']) / 3600
    tr1 = len(d1[d1['mode'] == 'nw-t']) / 3600
    # Compute total harvest and field time
    tw_tot = tw0 + tw1
    tf_tot = tf0 + tf1
    # Compute individual and overall time efficiencies
    et0 = tw0 / (tw0 + tp0 + tr0 + ts0)
    et1 = tw1 / (tw1 + tp1 + tr1 + ts1)
    Et = (tw0+tw1) / ((tw0+tw1) + (tp0+tp1) + (tr0+tr1) + (ts0+ts1))
    return [et0, et1, tw0, tw1, Et, tw_tot, tf_tot]

In [7]:
# Function for computing area capacity for each machine and for the field
def compute_area_stats(d0, d1, fs_poly):
    # Get only the working polygons
    d0w = d0.loc[d0['mode'] == 'w', 'geom_poly_cvg']
    d1w = d1.loc[d1['mode'] == 'w', 'geom_poly_cvg']
#     d0w = d0.loc[:, 'geom_poly_cvg']
#     d1w = d1.loc[:, 'geom_poly_cvg']
    # Get total field time
    t0 = len(d0)
    t1 = len(d1)
    # Get polygons
    poly_cvg0 = gpd.GeoSeries(d0w)
    poly_cvg1 = gpd.GeoSeries(d1w)
    # Get all polygons
    poly_cvg_all0 = gpd.GeoSeries(d0.loc[:, 'geom_poly_cvg'])
    poly_cvg_all1 = gpd.GeoSeries(d1.loc[:, 'geom_poly_cvg'])
    # Compute A_trv
    A_trv = (poly_cvg_all0.area.sum() + poly_cvg_all1.area.sum()) * 0.0002471052
    # Get individual covered area
    a_act0 = poly_cvg0.unary_union.area * 0.0002471052
    a_overlap_0 = poly_cvg0.unary_union.intersection(poly_cvg1.unary_union).area * 0.0002471052
    a_act0 = a_act0 - a_overlap_0 / 2
    a_act1 = poly_cvg1.unary_union.area * 0.0002471052
    a_overlap_1 = poly_cvg1.unary_union.intersection(poly_cvg0.unary_union).area * 0.0002471052
    a_act1 = a_act1 - a_overlap_1 / 2
    # Obtain the covered area polygon
    poly_cvg_union = unary_union([poly_cvg0.unary_union, poly_cvg1.unary_union])
    poly_cvg_actual = fs_poly.intersection(poly_cvg_union)
    # Get the actual covered area
#     A_act = poly_cvg_union.area * 0.0002471052
    A_act = poly_cvg_actual.area * 0.0002471052
    # Compute Eg
    Eg = A_act / A_trv * 100
    # Compute individual area capacity and overall area capacity
    ca0 = a_act0 / (t0 / 3600)
    ca1 = a_act1 / (t1 / 3600)
    Ca = A_act / ((t0 + t1) / 3600)
    # Compute coverage percentage
    cov_p0 = a_act0 / A_act * 100
    cov_p1 = a_act1 / A_act * 100
    
    return [a_act0, a_act1, A_act, A_trv, ca0, ca1, Ca, cov_p0, cov_p1, Eg]

In [8]:
report = {}

In [9]:
for k, v in data_maps.items():
    field_id = k
    print('Field: {}\n'.format(field_id))
    fs = fss[fss['Name'] == field_id].copy()
    fs = fs.reset_index(drop=True)
    proj = Proj(proj='utm', zone=13, ellps='WGS84', preserve_units=False)
    fs_poly = transform(lambda x, y, z: proj(x, y), fs['geometry'][0])
    M = []
    yy = []
    mm = []
    w_info = {}
    report[field_id] = []
    for kk, vv in v.items():
        year = kk
        yy.append(kk)
        machines = vv
        mm.append(vv)
        print('\tYear: {}\n'.format(year))
        with open(OPT_PICKLE_PATH + '-'.join([EVENT, '-'.join(machines), year, field_id, 'opt-res.pickle']), 'rb') as fh:
            opt_res = pickle.load(fh)
        xoff0, yoff0, xoff1, yoff1, w0, w1 = opt_res
        w_info[year] = [[machines[0], str(int(w0*3.28084))], [machines[1], str(int(w1*3.28084))]]
        w0 = w0 * 3.28084
        w1 = w1 * 3.28084
        print('\t\t{}: xoff = {:.2f}, yoff = {:.2f}, w = {:.2f}'.format(machines[0], xoff0, yoff0, w0))
        print('\t\t{}: xoff = {:.2f}, yoff = {:.2f}, w = {:.2f}'.format(machines[1], xoff1, yoff1, w1))
        # Load in data
        esw_pkl_names = ['-'.join([EVENT, mid, year, field_id, 'gps-esw.pickle']) for mid in machines]
        esw_df = []
        for n in esw_pkl_names:
            with open(ESW_PICKLE_PATH + n, 'rb') as fh:
                esw_df.append(pickle.load(fh))
        m0 = esw_df[0]
        m1 = esw_df[1]
        # Get rid of NaN nastiness
        m0 = m0.loc[(~m0['xs_s'].isna()) & (~m0['ys_s'].isna())].copy()
        m1 = m1.loc[(~m1['xs_s'].isna()) & (~m1['ys_s'].isna())].copy()
        m0 = m0.reset_index(drop=True).copy()
        m1 = m1.reset_index(drop=True).copy()
        print('\t\tComputing stats ...')
        # Concatenate two machines' data
        D = pd.concat([m0,m1], ignore_index=True)
        # Compute average harvest speed (mph)
        s_avg0 = m0.loc[m0['mode'] == 'w', 'speed'].median() * 2.236936
        s_avg1 = m1.loc[m1['mode'] == 'w', 'speed'].median() * 2.236936
        S_avg = D.loc[D['mode'] == 'w', 'speed'].median() * 2.236936
        # Compute average swath width (ft)
        esw_avg0 = m0.loc[m0['mode'] == 'w', 'esw'].mean() * 3.28084
        esw_avg1 = m1.loc[m1['mode'] == 'w', 'esw'].mean() * 3.28084
        Esw_avg = D.loc[D['mode'] == 'w', 'esw'].mean() * 3.28084
        # Compute time-related stats
        et0, et1, th0, th1, Et, tw_tot, tf_tot = compute_time_stats(m0, m1)
        # Compute area-related stats
        a_act0, a_act1, A_act, A_trv, ca0, ca1, Ca, cov_p0, cov_p1, Eg = compute_area_stats(m0, m1, fs_poly)
        report[field_id].append({
            year: {
                machines[0]: {
                    'sw': w0,
                    's_avg': s_avg0,
                    'esw_avg': esw_avg0,
                    'et': et0,
                    'th': th0,
                    'a_act': a_act0,
                    'ca': ca0,
                    'cov_p': cov_p0
                },
                machines[1]: {
                    'sw': w1,
                    's_avg': s_avg1,
                    'esw_avg': esw_avg1,
                    'et': et1,
                    'th': th1,
                    'a_act': a_act1,
                    'ca': ca1,
                    'cov_p': cov_p1
                },
                'overall': {
                    'S_avg': S_avg,
                    'Esw_avg': Esw_avg,
                    'Et': Et,
                    'Th': tw_tot,
                    'Tf': tf_tot,
                    'A_act': A_act,
                    'Ca': Ca,
                    'Eg': Eg
                }
            }
        })
        print('\t\tDone!\n')
#         print('\t\tComputing inst. parameters ...')
#         # Swath utilization factor
#         m0['U_ins'] = np.nan
#         m0['U_avg'] = np.nan
#         m1['U_ins'] = np.nan
#         m1['U_avg'] = np.nan
#         # Area capacity
#         m0['C_ins'] = np.nan
#         m0['C_avg'] = np.nan
#         m1['C_ins'] = np.nan
#         m1['C_avg'] = np.nan
#         # Compute both instantaenous U and C_a
#         m0_gdf = gpd.GeoDataFrame(m0, geometry='geom_poly_cvg')
#         m1_gdf = gpd.GeoDataFrame(m1, geometry='geom_poly_cvg')
#         # Compute instantaneous and avg E_f
#         # Start with m0 first
#         m0_gdf['U_ins'] = (m0_gdf['esw'] * 3.28084) / w0 
#         m0_gdf['U_avg'] = m0_gdf['U_ins'].cumsum() / range(1,len(m0_gdf)+1)
#         for idx in range(len(m0_gdf)):
#             if m0_gdf.loc[idx, 'geom_poly_cvg'] == None:
#                 continue
#             # Get the timestamp at the current index
#             ts_end = m0_gdf.loc[idx]['ts']
#             # Obtain the total area
#             area_tot = m0_gdf.loc[idx, 'geom_poly_cvg'].area
#             # Currently the actual area is the same as the total area
#             area_actual = area_tot
#             # Get the intersected area from the other machine
#             int_idx = m1_gdf[m1_gdf.loc[:, 'geom_poly_cvg'].intersects(m0_gdf.loc[idx, 'geom_poly_cvg'])]['ts'] < m0_gdf.loc[idx]['ts']
#             area_int = m1_gdf.loc[int_idx[int_idx == True].index, :].intersection(m0_gdf.loc[idx, 'geom_poly_cvg']).unary_union.area
#             area_actual -= area_int
#             if area_actual < 0:
#                 area_actual = 0
#             if area_actual > 40:
#                 area_actual = area_actual / 2 #HACK
#             # Compute instantaneous area capacity
#             m0_gdf.loc[idx, 'C_ins'] = area_actual * 0.00024711 / (1 / 3600)
#         m0_gdf['C_avg'] = m0_gdf['C_ins'].cumsum() / range(1,len(m0_gdf)+1)

#         # Do the same for m1
#         m1_gdf['U_ins'] = (m1_gdf['esw'] * 3.28084) / w1
#         m1_gdf['U_avg'] = m1_gdf['U_ins'].cumsum() / range(1,len(m1_gdf)+1)
#         for idx in range(len(m1_gdf)):
#             if m1_gdf.loc[idx, 'geom_poly_cvg'] == None:
#                 continue
#             # Get the timestamp at the current index
#             ts_end = m1_gdf.loc[idx]['ts']
#             # Obtain the total area
#             area_tot = m1_gdf.loc[idx, 'geom_poly_cvg'].area
#             # Currently the actual area is the same as the total area
#             area_actual = area_tot
#             # Get the intersected area from the other machine
#             int_idx = m0_gdf[m0_gdf.loc[:, 'geom_poly_cvg'].intersects(m1_gdf.loc[idx, 'geom_poly_cvg'])]['ts'] < m1_gdf.loc[idx]['ts']
#             area_int = m0_gdf.loc[int_idx[int_idx == True].index, :].intersection(m1_gdf.loc[idx, 'geom_poly_cvg']).unary_union.area
#             area_actual -= area_int
#             if area_actual < 0:
#                 area_actual = 0
#             if area_actual > 40:
#                 area_actual = area_actual / 2 #HACK
#             # Compute field capacity
#             m1_gdf.loc[idx, 'C_ins'] = area_actual * 0.00024711 / (1 / 3600)
#         m1_gdf['C_avg'] = m1_gdf['C_ins'].cumsum() / range(1,len(m1_gdf)+1)
#         print('\t\tDone!')
#         # Convert it to dataframe again
#         d0 = pd.DataFrame(m0_gdf)
#         d1 = pd.DataFrame(m1_gdf)
#         # Save it
#         print('\t\tSaving it ...')
#         d0[['ts', 'ts_local', 'mode', 'speed', 'xs_s', 'ys_s', 'esw', 'U_ins', 'C_ins', 'U_avg', 'C_avg']].to_hdf(\
#             OUTPUT_PATH + '-'.join([EVENT, machines[0], year, field_id, 'gps-eff.h5']), mode='w', key='df')
#         d1[['ts', 'ts_local', 'mode', 'speed', 'xs_s', 'ys_s', 'esw', 'U_ins', 'C_ins', 'U_avg', 'C_avg']].to_hdf(\
#             OUTPUT_PATH + '-'.join([EVENT, machines[1], year, field_id, 'gps-eff.h5']), mode='w', key='df')
#         print('\t\tDone!\n')

Field: f12

	Year: 072016

		2388: xoff = -1.80, yoff = 0.10, w = 30.00
		6088: xoff = -0.48, yoff = 0.04, w = 32.00
		Computing stats ...
		Done!

	Year: 072019

		2388: xoff = -0.89, yoff = 0.08, w = 32.00
		6088: xoff = -1.53, yoff = 0.15, w = 32.00
		Computing stats ...
		Done!

Field: f10

	Year: 072014

		6088: xoff = -0.17, yoff = 0.01, w = 30.00
		6130: xoff = -0.05, yoff = 0.01, w = 30.00
		Computing stats ...
		Done!

	Year: 072017

		2388: xoff = -1.42, yoff = 0.11, w = 30.00
		6088: xoff = -1.25, yoff = 0.09, w = 32.00
		Computing stats ...
		Done!

Field: f7

	Year: 072014

		6088: xoff = -0.40, yoff = 0.05, w = 30.00
		6130: xoff = -0.11, yoff = 0.02, w = 30.00
		Computing stats ...
		Done!

	Year: 072018

		7130: xoff = 0.16, yoff = 0.02, w = 30.00
		8240: xoff = -0.76, yoff = 0.02, w = 35.00
		Computing stats ...
		Done!

Field: f11

	Year: 072014

		6088: xoff = -0.04, yoff = 0.01, w = 30.00
		6130: xoff = -0.06, yoff = 0.01, w = 30.00
		Computing stats ...
		Done!

	Y

In [10]:
report

{'f12': [{'072016': {'2388': {'sw': 30.00000096,
     's_avg': 3.355404,
     'esw_avg': 24.568359339671773,
     'et': 0.8531930333817126,
     'th': 3.265833333333333,
     'a_act': 31.466251968813516,
     'ca': 8.220501240038363,
     'cov_p': 42.65341567590101},
    '6088': {'sw': 32.00131336,
     's_avg': 4.473872,
     'esw_avg': 25.816897545002185,
     'et': 0.8062085593731164,
     'th': 2.9722222222222223,
     'a_act': 42.328415958951815,
     'ca': 11.481487149806098,
     'cov_p': 57.37739348775993},
    'overall': {'S_avg': 3.914638,
     'Esw_avg': 25.16324551311593,
     'Et': 0.8301419488392725,
     'Th': 6.2380555555555555,
     'Tf': 7.514444444444445,
     'A_act': 73.77193941021659,
     'Ca': 9.817351097027197,
     'Eg': 65.05837714490225}}},
  {'072019': {'2388': {'sw': 32.00131336,
     's_avg': 3.355404,
     'esw_avg': 27.654145803808564,
     'et': 0.905988409529942,
     'th': 3.1266666666666665,
     'a_act': 36.50937684757278,
     'ca': 10.57902097965

In [11]:
# with open(REPORT_PATH + 'report.pickle', 'wb') as fp:
#     pickle.dump(report, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
# Sanity check
# fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,6))
# d0[d0['mode'] == 'w'].plot.scatter(x='xs_s', y='ys_s', s=1, c='g', ax=ax1)
# d0[d0['mode'] == 'nw-t'].plot.scatter(x='xs_s', y='ys_s', s=1, c='r', ax=ax1)
# d0[d0['mode'] == 'nw-p'].plot.scatter(x='xs_s', y='ys_s', s=1, c='royalblue', ax=ax1)
# d0[d0['mode'] == 'nw-s'].plot.scatter(x='xs_s', y='ys_s', s=1, c='orange', ax=ax1)
# d1[d1['mode'] == 'w'].plot.scatter(x='xs_s', y='ys_s', s=1, c='g', ax=ax2)
# d1[d1['mode'] == 'nw-t'].plot.scatter(x='xs_s', y='ys_s', s=1, c='r', ax=ax2)
# d1[d1['mode'] == 'nw-p'].plot.scatter(x='xs_s', y='ys_s', s=1, c='royalblue', ax=ax2)
# d1[d1['mode'] == 'nw-s'].plot.scatter(x='xs_s', y='ys_s', s=1, c='orange', ax=ax2)
# ax1.axis('equal')
# ax2.axis('equal')

In [13]:
# # Compute instantaneous and avg E_f
# d0_c_gdf['U_ins'] = d0_c_gdf['esw'] / w0
# d0_c_gdf['U_avg'] = d0_c_gdf['U_ins'].cumsum() / range(1,len(d0_c_gdf)+1)

# for idx in tqdm(range(len(d0_c_gdf))):
# #     if d0_c_gdf.loc[idx, 'mode'] != 'w':
# #         d0_c_gdf.loc[idx, 'C_ins'] = 0
# #         continue
#     if d0_c_gdf.loc[idx, 'geom_poly_cvg'] == None:
#         continue
#     # Get the timestamp at the current index
#     ts_end = d0_c_gdf.loc[idx]['ts']
#     # Obtain the total area
#     area_tot = d0_c_gdf.loc[idx, 'geom_poly_cvg'].area
#     # Currently the actual area is the same as the total area
#     area_actual = area_tot
#     # Get the intersected area from the other machine
#     int_idx = d1_c_gdf[d1_c_gdf.loc[:, 'geom_poly_cvg'].intersects(d0_c_gdf.loc[idx, 'geom_poly_cvg'])]['ts'] < d0_c_gdf.loc[idx]['ts']
#     area_int = d1_c_gdf.loc[int_idx[int_idx == True].index, :].intersection(d0_c_gdf.loc[idx, 'geom_poly_cvg']).unary_union.area
#     area_actual -= area_int
#     if area_actual < 0:
#         area_actual = 0
#     if area_actual > 40:
#         area_actual = 10 #HACK
#     # Compute instantaneous area capacity
#     d0_c_gdf.loc[idx, 'C_ins'] = area_actual * 0.00024711 / (1 / 3600)
    
# d0_c_gdf['C_avg'] = d0_c_gdf['C_ins'].cumsum() / range(1,len(d0_c_gdf)+1)

In [14]:
# # Compute instantaneous and avg E_f
# d1_c_gdf['U_ins'] = d1_c_gdf['esw'] / w1
# d1_c_gdf['U_avg'] = d1_c_gdf['U_ins'].cumsum() / range(1,len(d1_c_gdf)+1)

# for idx in tqdm(range(len(d1_c_gdf))):
# #     if d1_c_gdf.loc[idx, 'mode'] != 'w':
# #         d1_c_gdf.loc[idx, 'C_ins'] = 0
# #         continue
#     if d1_c_gdf.loc[idx, 'geom_poly_cvg'] == None:
#         continue
#     # Get the timestamp at the current index
#     ts_end = d1_c_gdf.loc[idx]['ts']
#     # Obtain the total area
#     area_tot = d1_c_gdf.loc[idx, 'geom_poly_cvg'].area
#     # Currently the actual area is the same as the total area
#     area_actual = area_tot
#     # Get the intersected area from the other machine
#     int_idx = d0_c_gdf[d0_c_gdf.loc[:, 'geom_poly_cvg'].intersects(d1_c_gdf.loc[idx, 'geom_poly_cvg'])]['ts'] < d1_c_gdf.loc[idx]['ts']
#     area_int = d0_c_gdf.loc[int_idx[int_idx == True].index, :].intersection(d1_c_gdf.loc[idx, 'geom_poly_cvg']).unary_union.area
#     area_actual -= area_int
#     if area_actual < 0:
#         area_actual = 0
#     if area_actual > 50:
#         area_actual = 10 #HACK
#     # Compute field capacity
#     d1_c_gdf.loc[idx, 'C_ins'] = area_actual * 0.00024711 / (1 / 3600)
    
# d1_c_gdf['C_avg'] = d1_c_gdf['C_ins'].cumsum() / range(1,len(d1_c_gdf)+1)